Implement a Hopfield Network, and perform the basic analysis below:

1. Store Implement a Python function which is able to store a memory in a Hopfield network.
2. Recall Implement a Python function which runs the recall algorithm for an initial (corrupted) memory. Make sure to include a stopping criteria.
3. Capacity Write a function to determine the capacity for a Hopfield network of a given size. This function should have a size parameter, and then initialize an empty Hopfield network. It must then store a random memory and test whether the network is able to recall the correct memory if given a pattern corrupted by a single bit. The function must then return the largest number of memories that were stored and successfully recalled.
4. Scaling Behavior Produce a plot showing capacity as a function of network size. What is the scaling behavior?

In [7]:
import numpy as np
from sklearn.datasets import load_digits

#Let's use MNIST digits as memories
def load_2digits(a, b):
    '''
    Returns array X with data and
    array y with targets 
    for two digits a, b in MNIST
    '''
    #Load entire data set
    digits = load_digits()
    y = digits.target
    
    #Extract desired subset
    indices = np.where((y == a) | (y == b))[0]
    X = digits.data[indices]
    y = digits.target[indices]
    
    return X, y

X, y = load_2digits(0,1)

In [ ]:
#Let's take one 0 and one 1 to store as memories
print(y[0], y[1])
zero = [1 if bit>0 else -1 for bit in X[0]]
one = [1 if bit>0 else -1 for bit in X[1]]

memories = np.array([zero, one])
memories.shape

In [10]:
def store(memories):
    number, size = memories.shape
    
    #Initialize Weight matrix
    W = np.zeros(shape=(size, size))
    
    for memory in memories:
        W += np.outer(memory, memory)
    
    #Enforce zero for self-weights
    W[np.diag_indices(size)] = 0
    return W/number

In [11]:
def recall(W, memories):
    sgn = np.vectorize(lambda x: -1 if x<0 else +1)
    for _ in range(5):        
        memories = sgn(np.dot(memories,W))
    return memories

In [24]:
def capacity(size):
    successful_recalls = 0
    while True:
        memories = np.array([np.random.randint(0, 2, (size, size)) for _ in range(successful_recalls+1)])
        print(memories.shape)
        #Make corrupted version differing by 1 bit
        corrupted_memories = memories
        for i in range(len(corrupted_memories)):
            corrupted_memories[i][np.random.randint(0, size-1)][np.random.randint(0, size-1)] *= -1
        
        W = store(memories)
        recalls = recall(W, corrupted_memories)
        for i in range(len(memories)):
            if not np.array_equal(recalls[i], memories[i]):
                break
                
        successful_recalls += 1
            
    return successful_recalls

In [13]:
#Xs = [x for x in range(2, 6)]
#Ys = [capacity(x) for x in Xs]

KeyboardInterrupt: 

In [25]:
# Kevin
class HopfieldNetwork(object):
    
    def __init__(self, neurons):
        self.W = [[0] * neurons for _ in range(neurons)]
        
    def train(self, training_data):
        for i in range(len(self.W)):
            for j in range(len(self.W[0])):
                if i == j or self.W[i][j] != 0:
                    continue
                
                w = 0
                for n in range(len(training_data)):
                    w += training_data[n][i] * training_data[n][j]
                    
                self.W[i][j] = w / float(len(training_data))
                self.W[j][i] = self.W[i][j]
    
    def predict(self, prediction, max_iter):
        res = list(prediction)
        
        for _ in range(max_iter):
            for j in range(len(res)):
                res[j] = 1 if sum([self.W[i][j] * res[i] for i in range(len(self.W[j]))]) > 0 else -1
                
        return res

import random

def getCapacity(size):
    c = 1
    while True:
        flag = True
        seen = set()
        train_set = []
        
        for _ in range(c):
            temp = [random.choice([-1, 1]) for _ in range(size)]
            while tuple(temp) in seen:
                temp = [random.choice([-1, 1]) for _ in range(size)]
            train_set.append(temp)
            seen.add(tuple(temp))
        
        a = HopfieldNetwork(size)
        a.train(train_set)
        
        test_set = []
        for instance in train_set:
            temp = list(instance)
            ind = random.randint(0, len(temp) - 1)
            temp[ind] = -temp[ind]
            test_set.append(temp)
            
        for i in range(len(test_set)):
            if a.predict(test_set[i], 20) != train_set[i]:
                return c
        
        c += 1

import numpy as np

cap = []
for iteration in range(5, 100, 5):
    cap.append(np.mean([getCapacity(iteration) for _ in range(5)]))

import matplotlib.pyplot as plt

plt.plot(range(5, 100, 5), cap)
plt.xlabel('Network Size')
plt.ylabel('Capacity')
plt.show()

(1, 2, 2)


ValueError: too many values to unpack (expected 2)

In [ ]:
#Juan

import numpy as np

class Hopfield_Network:
    def __init__(self,N):
        self.size = N
        self.W = np.zeros((N,N))
        self.x = -np.ones(N) 
        #P = number of learned patterns
        self.P = 0
        self.learned_patterns = []
    
    def learn_pattern(self,p,e=0.1):
        self.P+=1
        self.learned_patterns.append(p)
        p = np.array([p])
        W = p.T.dot(p)
        np.fill_diagonal(W,0)
        self.W = self.W+e*W
        
    def set_inputs(self,x):
        self.x = x
        
    def update_one_neuron(self,i):
        activation = lambda f : -1+2*(f>0)
        b=0 #bias
        self.x[i] = activation(np.dot(self.W[i],self.x) + b)
    
    def new_output_neuron(self,i):
        activation = lambda f : -1+2*(f>0)
        b=0 #bias
        return activation(np.dot(self.W[i],self.x) + b)
    
    def sync_update_all(self):
        for i in range(self.size):
            self.update_one_neuron(i)
        
    def async_update_all(self):
        updated_x = self.x
        for i in range(self.size):
            updated_x[i] = self.new_output_neuron(i)
        self.x = updated_x
    
    def recall(self, print_ = False):
        old_config = self.x
        new_config = np.zeros(self.size)
        if print_:
            for r in range(5):
                    print(self.x[5*r:5*r+5])
        while not (old_config == new_config).all():
            old_config = new_config
            self.async_update_all()
            new_config = self.x
            if print_:
                for r in range(5):
                    print(self.x[5*r:5*r+5])

def test_capacity_HN(size):
    HN = Hopfield_Network(size)
    for i in range(50):
        p = np.random.randint(0,2,size)
        for j in range(size):
            if p[j] == 0:
                p[j] = -1
        HN.learn_pattern(p)
        ind = int(np.random.randint(0,size,1))
        p_recall = p
        p_recall[ind]*=-1
        HN.set_inputs(p_recall)
        HN.recall(print_=True)
        if not (HN.x == p).all():
            print(i)
            break

test_capacity()